# **Durée de retour à l'emploi : phase exploratoire**

In [1]:
# nécessité d'importer la librairie xlrd pour permettre à Python de lire Excel (les données INSEE ne sont pas dispobinle en csv)
!pip install xlrd

In [2]:
!pip install openpyxl
# également demandé par Python pour lire Excel

In [3]:
import pandas as pd
import xlrd
import openpyxl
import numpy as np

## 1. Constitution de la BDD OPEN DATA

4 fichiers sont nécessaires à la constitution de cette df :
- 2 indicateurs de dynamisme du marché du travail : taux de chômage et indice de concentration de l'emploi
- 2 codes pour opérer les jointures : code INSEE de la commune (jointure avec les codes postaux) et de la Zone d'emploi (jointure avec les codes communes et le taux de chômage)

**Chargement du fichier de correspondance Code INSEE - Code postal**
- Source : La Poste sur Open Data

> *La base officielle des codes postaux est un jeu de données qui fournit la correspondance entre les codes postaux et les codes INSEE des communes, de France (métropole et DOM), des TOM, ainsi que de MONACO. Le fichier CSV a été mis à jour le 21 mars 2022*

- URL : https://www.data.gouv.fr/fr/datasets/base-officielle-des-codes-postaux/#resources

In [4]:
# IMPOSSIBLE DE CONVERTIR LE CODE POSTAL EN DONNEE CATEGORIELLE malgré reformatage dans excel ALORS QUE ça a marché pour le code commune
#communes_dtypes = {'code_commune_insee':str,'nom_de_la_commune':str, 'code_postal':str}
#communes_df = pd.read_excel(r"C:\Users\emman\Documents\4_Diplômes\2022_DESU-AMU\0_Projet\projet_desu_tre\data\external\laposte_hexasmal_4.xlsx")
#communes_df['code_postal'] = communes_df['code_postal'].astype(str) # garder le format string pour la colonne cp (code postal)

In [5]:
communes_df = pd.read_excel(r"C:\Users\emman\Documents\4_Diplômes\2022_DESU-AMU\0_Projet\projet_desu_tre\data\external\laposte_hexasmal_4.xlsx")
communes_df['cp1'] = communes_df['cp1'].astype(str) # garder le format string pour la colonne cp (code postal)
communes_df

,code_commune_insee,nom_de_la_commune,code_postal,ligne_5,libelle_d_acheminement,coordonnees_gps,cp1,cp2,cp3,cp4
0,01053,BOURG EN BRESSE,1000.0,NaN,BOURG EN BRESSE,"46.205013838,5.245593605",1000.0,1000.0,1000,1000
1,01053,BOURG EN BRESSE,1000.0,BROU,BOURG EN BRESSE,"46.205013838,5.245593605",1000.0,1000.0,1000,1000
2,01344,ST DENIS LES BOURG,1000.0,NaN,ST DENIS LES BOURG,"46.211139007,5.184567751",1000.0,1000.0,1000,1000
3,01165,FRANCHELEINS,1090.0,AMAREINS,FRANCHELEINS,"46.073642487,4.812560271",1090.0,1090.0,1090,1090
4,01165,FRANCHELEINS,1090.0,NaN,FRANCHELEINS,"46.073642487,4.812560271",1090.0,1090.0,1090,1090
...,...,...,...,...,...,...,...,...,...,...
39188,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,98883,98883
39189,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,98884,98884
39190,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,98885,98885
39191,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,98889,98889


In [6]:
communes_df.shape

(39193, 10)

In [7]:
# modification et préparation de la table communes_df : en-têtes renommées
communes_df = communes_df.rename(columns = {'code_postal' : 'cp', 'code_commune_insee':'code_commune', 'nom_de_la_commune' : 'commune'})
communes_df = communes_df[['cp', 'code_commune', 'commune']] # ne garder que les colonnes utiles
communes_df

,cp,code_commune,commune
0,1000.0,01053,BOURG EN BRESSE
1,1000.0,01053,BOURG EN BRESSE
2,1000.0,01344,ST DENIS LES BOURG
3,1090.0,01165,FRANCHELEINS
4,1090.0,01165,FRANCHELEINS
...,...,...,...
39188,NaN,NaN,NaN
39189,NaN,NaN,NaN
39190,NaN,NaN,NaN
39191,NaN,NaN,NaN


In [8]:
#vérifier le nombre de valeurs manquantes dans chaque colonne
communes_df.isna().sum()

cp              267
code_commune    267
commune         267
dtype: int64

In [9]:
# conversion des données mixtes de la colonne "code commune" et "code postal" en string afin de pouvoir faire le merge
communes_df['code_commune'] = communes_df['code_commune'].astype(str)

In [10]:
#vérifier les types des colonnes : 
communes_df.dtypes

cp              float64
code_commune     object
commune          object
dtype: object

**Chargement du fichier base des zones d'emploi Code INSEE - ZE depuis INSEE**
- Source : INSEE - composition communale des zones d'emploi (au format XLSX)
> *Base des zones d'emploi 2020 au 1ᵉʳ janvier 2022. Permet de rattacher chaque commune à sa zone d'emploi, car le taux de chômage actualisé est disponible par zone d'emploi et non par commune*
- URL : https://www.insee.fr/fr/information/4652957

In [11]:
ze_df = pd.read_excel(r"C:\Users\emman\Documents\4_Diplômes\2022_DESU-AMU\0_Projet\projet_desu_tre\data\external\ZE2020_au_01-01-2022.xlsx", skiprows = 4, header = 1, sheet_name = 'Composition_communale')
ze_df

,CODGEO,LIBGEO,ZE2020,LIBZE2020,ZE2020_PARTIE_REG,DEP,REG
0,01001,L'Abergement-Clémenciat,8405,Bourg en Bresse,NaN,01,84
1,01002,L'Abergement-de-Varey,8405,Bourg en Bresse,NaN,01,84
2,01004,Ambérieu-en-Bugey,8405,Bourg en Bresse,NaN,01,84
3,01005,Ambérieux-en-Dombes,8434,Villefranche-sur-Saône,NaN,01,84
4,01006,Ambléon,8404,Belley,NaN,01,84
...,...,...,...,...,...,...,...
34950,97613,M'Tsangamouji,601,Mayotte,NaN,976,6
34951,97614,Ouangani,601,Mayotte,NaN,976,6
34952,97615,Pamandzi,601,Mayotte,NaN,976,6
34953,97616,Sada,601,Mayotte,NaN,976,6


In [12]:
# modification et préparation de la table ze_df : en-têtes renommées
ze_df = ze_df.rename(columns = {"CODGEO":'code_commune2', 
                                          "LIBGEO" : 'commune2',  
                                          "ZE2020" : 'code_ze',
                                         "LIBZE2020" : 'ze'})
ze_df = ze_df[["code_commune2", "commune2", "code_ze", "ze"]] # on ne garde que les colonnes utiles 
ze_df.code_ze = ze_df.code_ze.astype(str) # fixer le code_ze en format STRING
ze_df

,code_commune2,commune2,code_ze,ze
0,01001,L'Abergement-Clémenciat,8405,Bourg en Bresse
1,01002,L'Abergement-de-Varey,8405,Bourg en Bresse
2,01004,Ambérieu-en-Bugey,8405,Bourg en Bresse
3,01005,Ambérieux-en-Dombes,8434,Villefranche-sur-Saône
4,01006,Ambléon,8404,Belley
...,...,...,...,...
34950,97613,M'Tsangamouji,601,Mayotte
34951,97614,Ouangani,601,Mayotte
34952,97615,Pamandzi,601,Mayotte
34953,97616,Sada,601,Mayotte


In [13]:
#vérifier le nombre de valeurs manquantes dans chaque colonne
ze_df.isna().sum()

code_commune2    0
commune2         0
code_ze          0
ze               0
dtype: int64

In [14]:
ze_df.dtypes

code_commune2    object
commune2         object
code_ze          object
ze               object
dtype: object

In [15]:
# conversion des données mixtes de la colonne "code commune" en string afin de pouvoir faire le merge
ze_df['code_commune2'] = ze_df['code_commune2'].astype(str)

In [16]:
#vérification du type des colonnes
ze_df.dtypes

code_commune2    object
commune2         object
code_ze          object
ze               object
dtype: object

**MERGE sur les 2 premiers JDD CP et codes communes et ZE**
> *Constitution d'un seul jeu avec toutes les informations CP, code commune et nom de la commune, code ZE et nom de la ZE. Dans un second temps, on enlevera les colonnes inutiles (gardées pour vérification).*

> *ATTENTION : le 1er JDD comprenait 39 193 lignes (codes postaux) : suppression depuis Excel des 267 codes postaux 98XXX (Collectivités d’outre-mer non dotées du statut de collectivité territoriale cad non habitées de façon permanente de types : terres australes et antartiques) + MONACO (99) + 5 codes 97XXX non répertoriés > il a donc été réduit à 38926 lignes

> Le second en a uniquement 34 955 (codes communes et codes ZE). Il faudra donc vérifier quels codes postaux n'ont pas de codes INSEE (par hypothèse les CP d'arrondissements à Paris, Marseille, Lyon)

**Choix du MERGE : on privilégie le JDD le plus complet : sur les communes**
> Le JDD sur les ZE avec 34955 lignes semble suffisant. Si on réalise le MERGE depuis le JDD avec 38926 lignes, on se retrouve avec 57 lignes avec des données manquantes : les arrondissements de Paris, Marseille et Lyon.

> A l'inverse, le MERGE sur le JDD des ZE avec 34 955 lignes n'a que 3 données manquantes, correspondant à l'équivalent des libéllée et codes de ZE pour Paris, Marseille et Lyon. Cela nous permettra de compléter la donnée que pour ces lignes.

> Les 2 autres JDD utilisés ensuite jointent sur la ZE (taux de chômage) et le code commune sans les arrondissements.

In [17]:
#MERGE des 2 dataframe à partir de la clé code_commune = code_commune2 avec JDD_ze à gauche (34 955) où il y en a moins que dans le JDD_commune (38 931)
INSEE_df = pd.merge(ze_df, communes_df, how = "left", left_on="code_commune2", right_on="code_commune")
print (INSEE_df) #vérifier le nombre de valeurs manquantes (null ou NA) dans chaque colonne
INSEE_df.isna().sum()

      code_commune2                 commune2 code_ze                      ze  \
0             01001  L'Abergement-Clémenciat    8405         Bourg en Bresse   
1             01002    L'Abergement-de-Varey    8405         Bourg en Bresse   
2             01004        Ambérieu-en-Bugey    8405         Bourg en Bresse   
3             01005      Ambérieux-en-Dombes    8434  Villefranche-sur-Saône   
4             01006                  Ambléon    8404                  Belley   
...             ...                      ...     ...                     ...   
38867         97614                 Ouangani     601                 Mayotte   
38868         97615                 Pamandzi     601                 Mayotte   
38869         97616                     Sada     601                 Mayotte   
38870         97617                 Tsingoni     601                 Mayotte   
38871         97617                 Tsingoni     601                 Mayotte   

            cp code_commune            

code_commune2    0
commune2         0
code_ze          0
ze               0
cp               3
code_commune     3
commune          3
dtype: int64

In [18]:
# identification et visualisation des 3 lignes avec des valeurs manquantes
INSEE_df[INSEE_df.code_commune.isna()]

,code_commune2,commune2,code_ze,ze,cp,code_commune,commune
4750,13055,Marseille,9312,Marseille,NaN,NaN,NaN
30158,69123,Lyon,8421,Lyon,NaN,NaN,NaN
32574,75056,Paris,1109,Paris,NaN,NaN,NaN


> Les données manquantes sont les 3 communes avec arrondissements

In [19]:
# Affiche le nombre de valeurs présentes dans chaque colonne
INSEE_df.count()

code_commune2    38872
commune2         38872
code_ze          38872
ze               38872
cp               38869
code_commune     38869
commune          38869
dtype: int64

In [20]:
#MERGE inversé des 2 dataframeq à partir de la clé code_commune2 = code_commune avec JDD_commune (38 931) à gauche car il en a plus que JDD_ze (34 955) 
INSEE1_df = pd.merge(communes_df, ze_df, how = "left", left_on="code_commune", right_on="code_commune2")
print (INSEE1_df) #vérifier le nombre de valeurs manquantes (null ou NA) dans chaque colonne
INSEE1_df.isna().sum()

           cp code_commune             commune code_commune2  \
0      1000.0        01053     BOURG EN BRESSE         01053   
1      1000.0        01053     BOURG EN BRESSE         01053   
2      1000.0        01344  ST DENIS LES BOURG         01344   
3      1090.0        01165        FRANCHELEINS         01165   
4      1090.0        01165        FRANCHELEINS         01165   
...       ...          ...                 ...           ...   
39188     NaN          nan                 NaN           NaN   
39189     NaN          nan                 NaN           NaN   
39190     NaN          nan                 NaN           NaN   
39191     NaN          nan                 NaN           NaN   
39192     NaN          nan                 NaN           NaN   

                    commune2 code_ze                      ze  
0            Bourg-en-Bresse    8405         Bourg en Bresse  
1            Bourg-en-Bresse    8405         Bourg en Bresse  
2      Saint-Denis-lès-Bourg    8405      

cp               267
code_commune       0
commune          267
code_commune2    324
commune2         324
code_ze          324
ze               324
dtype: int64

**Mise à jour du JDD INSEE1 : complétion des valeurs manquantes**

Il s'agira d'abord d'identifier les valeurs manquantes du JDD INSEE1 à l'aide du JDD INSEE, puis de remplacer les NaN par les valeurs adéquates

In [21]:
# identification et visualisation des 57 lignes avec des valeurs manquantes
INSEE1_df[INSEE1_df.code_commune2.isna()]

,cp,code_commune,commune,code_commune2,commune2,code_ze,ze
4670,13001.0,13201,MARSEILLE 01,NaN,NaN,NaN,NaN
4671,13002.0,13202,MARSEILLE 02,NaN,NaN,NaN,NaN
4672,13003.0,13203,MARSEILLE 03,NaN,NaN,NaN,NaN
4673,13004.0,13204,MARSEILLE 04,NaN,NaN,NaN,NaN
4674,13005.0,13205,MARSEILLE 05,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
39188,NaN,nan,NaN,NaN,NaN,NaN,NaN
39189,NaN,nan,NaN,NaN,NaN,NaN,NaN
39190,NaN,nan,NaN,NaN,NaN,NaN,NaN
39191,NaN,nan,NaN,NaN,NaN,NaN,NaN


> Les 57 lignes avec données manquantes sont les arrondissements de Marseille, Lyon et Paris, et les codes géographiques 97XXX

In [22]:
# identification et visualisation des 3 valeurs de destination : codes commune2 + nom2 + code ZE + ze génériques pour Marseille, Lyon et Paris
INSEE_df[INSEE_df.code_commune.isna()]

,code_commune2,commune2,code_ze,ze,cp,code_commune,commune
4750,13055,Marseille,9312,Marseille,NaN,NaN,NaN
30158,69123,Lyon,8421,Lyon,NaN,NaN,NaN
32574,75056,Paris,1109,Paris,NaN,NaN,NaN


In [23]:
#vérification de l'index des lignes Marseille (RAPPEL : borne extérieure exclue du slice)
INSEE1_df[4832:4858]

,cp,code_commune,commune,code_commune2,commune2,code_ze,ze
4832,13760.0,13091,ST CANNAT,13091,Saint-Cannat,9301,Aix-en-Provence
4833,13770.0,13113,VENELLES,13113,Venelles,9301,Aix-en-Provence
4834,13780.0,13030,CUGES LES PINS,13030,Cuges-les-Pins,9312,Marseille
4835,13780.0,83105,RIBOUX,83105,Riboux,9312,Marseille
4836,13790.0,13025,CHATEAUNEUF LE ROUGE,13025,Châteauneuf-le-Rouge,9301,Aix-en-Provence
4837,13790.0,13072,PEYNIER,13072,Peynier,9301,Aix-en-Provence
4838,13790.0,13072,PEYNIER,13072,Peynier,9301,Aix-en-Provence
4839,13790.0,13087,ROUSSET,13087,Rousset,9301,Aix-en-Provence
4840,13800.0,13047,ISTRES,13047,Istres,9313,Martigues-Salon
4841,13810.0,13034,EYGALIERES,13034,Eygalières,9306,Cavaillon


In [24]:
#Traitement des valeurs manquantes : sur le JDD INSEE1 > affecter le code commune et ze aux arrondissements de Marseille au lieu des valeurs NaN
INSEE1_df.loc[4832:4858,"code_commune2"] = "13055" #affecte le code commune 2
INSEE1_df.loc[4832:4858,"commune2"] = "Marseille" #affecte le commune 2
INSEE1_df.loc[4832:4858,"code_ze"] = "9312" #affecte le code ze
INSEE1_df.loc[4832:4858,"ze"] = "Marseille" #affecte le nom de la ze
INSEE1_df[4832:4858] #affiche les lignes avec les nouvelles valeurs

,cp,code_commune,commune,code_commune2,commune2,code_ze,ze
4832,13760.0,13091,ST CANNAT,13055,Marseille,9312,Marseille
4833,13770.0,13113,VENELLES,13055,Marseille,9312,Marseille
4834,13780.0,13030,CUGES LES PINS,13055,Marseille,9312,Marseille
4835,13780.0,83105,RIBOUX,13055,Marseille,9312,Marseille
4836,13790.0,13025,CHATEAUNEUF LE ROUGE,13055,Marseille,9312,Marseille
4837,13790.0,13072,PEYNIER,13055,Marseille,9312,Marseille
4838,13790.0,13072,PEYNIER,13055,Marseille,9312,Marseille
4839,13790.0,13087,ROUSSET,13055,Marseille,9312,Marseille
4840,13800.0,13047,ISTRES,13055,Marseille,9312,Marseille
4841,13810.0,13034,EYGALIERES,13055,Marseille,9312,Marseille


In [25]:
#vérification de l'index des lignes Lyon (RAPPEL : borne extérieure exclue du slice)
INSEE1_df[29964:29973]

,cp,code_commune,commune,code_commune2,commune2,code_ze,ze
29964,68630.0,68026,BENNWIHR,68026,Bennwihr,4405,Colmar
29965,68630.0,68209,MITTELWIHR,68209,Mittelwihr,4405,Colmar
29966,68640.0,68087,FELDBACH,68087,Feldbach,4418,Saint-Louis
29967,68640.0,68221,MUESPACH,68221,Muespach,4418,Saint-Louis
29968,68640.0,68221,MUESPACH,68221,Muespach,4418,Saint-Louis
29969,68640.0,68222,MUESPACH LE HAUT,68222,Muespach-le-Haut,4418,Saint-Louis
29970,68640.0,68273,RIESPACH,68273,Riespach,4418,Saint-Louis
29971,68640.0,68325,STEINSOULTZ,68325,Steinsoultz,4418,Saint-Louis
29972,68640.0,68355,WALDIGHOFEN,68355,Waldighofen,4418,Saint-Louis


In [26]:
#Traitement des valeurs manquantes : sur le JDD INSEE1 > affecter le code commune et ze aux arrondissements de Lyon au lieu des valeurs NaN
INSEE1_df.loc[29964:29973,"code_commune2"] = "69123" #affecte le code commune 2
INSEE1_df.loc[29964:29973,"commune2"] = "Lyon" #affecte le commune 2
INSEE1_df.loc[29964:29973,"code_ze"] = "8421" #affecte le code ze
INSEE1_df.loc[29964:29973,"ze"] = "Lyon" #affecte le nom de la ze
INSEE1_df[29964:29973] #affiche les lignes avec les nouvelles valeurs

,cp,code_commune,commune,code_commune2,commune2,code_ze,ze
29964,68630.0,68026,BENNWIHR,69123,Lyon,8421,Lyon
29965,68630.0,68209,MITTELWIHR,69123,Lyon,8421,Lyon
29966,68640.0,68087,FELDBACH,69123,Lyon,8421,Lyon
29967,68640.0,68221,MUESPACH,69123,Lyon,8421,Lyon
29968,68640.0,68221,MUESPACH,69123,Lyon,8421,Lyon
29969,68640.0,68222,MUESPACH LE HAUT,69123,Lyon,8421,Lyon
29970,68640.0,68273,RIESPACH,69123,Lyon,8421,Lyon
29971,68640.0,68325,STEINSOULTZ,69123,Lyon,8421,Lyon
29972,68640.0,68355,WALDIGHOFEN,69123,Lyon,8421,Lyon


In [27]:
#vérification de l'index des lignes Paris (RAPPEL : borne extérieure exclue du slice)
INSEE1_df[32189:32210]

,cp,code_commune,commune,code_commune2,commune2,code_ze,ze
32189,73460.0,73297,TOURNON,73297,Tournon,8413,La Tarentaise
32190,73460.0,73312,VERRENS ARVEY,73312,Verrens-Arvey,8413,La Tarentaise
32191,73470.0,73027,AYN,73027,Ayn,8407,Chambéry
32192,73470.0,73122,GERBAIX,73122,Gerbaix,8407,Chambéry
32193,73470.0,73152,MARCIEUX,73152,Marcieux,8407,Chambéry
32194,73470.0,73184,NANCES,73184,Nances,8407,Chambéry
32195,73470.0,73191,NOVALAISE,73191,Novalaise,8407,Chambéry
32196,73480.0,73040,BESSANS,73040,Bessans,8411,La Maurienne
32197,73480.0,73047,BONNEVAL SUR ARC,73047,Bonneval-sur-Arc,8411,La Maurienne
32198,73480.0,73290,VAL CENIS,73290,Val-Cenis,8411,La Maurienne


In [28]:
#Traitement des valeurs manquantes : sur le JDD INSEE1 > affecter le code commune et ze aux arrondissements de Paris au lieu des valeurs NaN
INSEE1_df.loc[32189:32210,"code_commune2"] = "75056" #affecte le code commune 2
INSEE1_df.loc[32189:32210,"commune2"] = "Paris" #affecte le commune 2
INSEE1_df.loc[32189:32210,"code_ze"] = "1109" #affecte le code ze
INSEE1_df.loc[32189:32210,"ze"] = "Paris" #affecte le nom de la ze
INSEE1_df[32189:32210] #affiche les lignes avec les nouvelles valeurs

,cp,code_commune,commune,code_commune2,commune2,code_ze,ze
32189,73460.0,73297,TOURNON,75056,Paris,1109,Paris
32190,73460.0,73312,VERRENS ARVEY,75056,Paris,1109,Paris
32191,73470.0,73027,AYN,75056,Paris,1109,Paris
32192,73470.0,73122,GERBAIX,75056,Paris,1109,Paris
32193,73470.0,73152,MARCIEUX,75056,Paris,1109,Paris
32194,73470.0,73184,NANCES,75056,Paris,1109,Paris
32195,73470.0,73191,NOVALAISE,75056,Paris,1109,Paris
32196,73480.0,73040,BESSANS,75056,Paris,1109,Paris
32197,73480.0,73047,BONNEVAL SUR ARC,75056,Paris,1109,Paris
32198,73480.0,73290,VAL CENIS,75056,Paris,1109,Paris


In [29]:
#vérifier que INSEE1_df a toujours le même nombre de lignes (38926)
INSEE1_df
#puis ²afficher le nombre de valeurs manquantes
INSEE1_df.isna().sum()

cp               267
code_commune       0
commune          267
code_commune2    324
commune2         324
code_ze          324
ze               324
dtype: int64

In [30]:
INSEE1_df.dtypes

cp               float64
code_commune      object
commune           object
code_commune2     object
commune2          object
code_ze           object
ze                object
dtype: object

In [31]:
INSEE1_df

,cp,code_commune,commune,code_commune2,commune2,code_ze,ze
0,1000.0,01053,BOURG EN BRESSE,01053,Bourg-en-Bresse,8405,Bourg en Bresse
1,1000.0,01053,BOURG EN BRESSE,01053,Bourg-en-Bresse,8405,Bourg en Bresse
2,1000.0,01344,ST DENIS LES BOURG,01344,Saint-Denis-lès-Bourg,8405,Bourg en Bresse
3,1090.0,01165,FRANCHELEINS,01165,Francheleins,8434,Villefranche-sur-Saône
4,1090.0,01165,FRANCHELEINS,01165,Francheleins,8434,Villefranche-sur-Saône
...,...,...,...,...,...,...,...
39188,NaN,nan,NaN,NaN,NaN,NaN,NaN
39189,NaN,nan,NaN,NaN,NaN,NaN,NaN
39190,NaN,nan,NaN,NaN,NaN,NaN,NaN
39191,NaN,nan,NaN,NaN,NaN,NaN,NaN


**Ajout du taux de chômage au JDD INSEE1 (constitution du JDD INSEE2)**

- Source : INSEE.fr

Dans un 1er temps, import du JDD de l'INSEE avec les taux de chômage actualisé au 4ème trimeste 2021

> *Le JDD en format Excel comprend les taux de chômage localisés par zone d'emploi, en moyenne trimestrielle, en France hors Mayotte et Guyane.*

- URL : https://www.insee.fr/fr/statistiques/1893230

Le fichier contient 5 lignes de métadonnées à supprimer et seules les colonnes A (ZE2020 = numéro de la ZE), B (LIBZE2020 = nom de la ZE) et CB (2021-T4 = taux de chômage au 4ème trimestre 2021) sont intéressantes à garder

In [32]:
# import du fichier avec suppression des métadonnées
chomage_df = pd.read_excel(r"C:\Users\emman\Documents\4_Diplômes\2022_DESU-AMU\0_Projet\projet_desu_tre\data\external\chomage-zone-t1-2003-t4-2021.xlsx",  skiprows = 4, header = 1, sheet_name = 'txcho_ze')
chomage_df

,ZE2020,LIBZE2020,REG,LIBREG,2003-T1,2003-T2,2003-T3,2003-T4,2004-T1,2004-T2,...,2019-T3,2019-T4,2020-T1,2020-T2,2020-T3,2020-T4,2021-T1,2021-T2,2021-T3,2021-T4
0,51,Alençon,0,INTERRÉGIONAL,7.2,7.5,7.3,7.5,7.7,7.4,...,8.2,8.1,7.7,7.1,8.7,7.6,7.4,7.5,7.7,7.0
1,52,Arles,0,INTERRÉGIONAL,10.8,11.0,10.7,11.1,11.2,10.8,...,11.1,10.7,10.4,9.8,11.8,10.3,10.3,10.7,10.5,9.9
2,53,Avignon,0,INTERRÉGIONAL,9.4,9.6,9.6,9.7,9.9,9.7,...,10.6,10.3,9.8,9.0,11.3,10.1,10.1,10.1,10.3,9.6
3,54,Beauvais,0,INTERRÉGIONAL,7.1,7.3,7.3,7.8,7.9,7.8,...,8.6,8.4,8.1,7.6,9.3,8.1,8.0,8.1,8.3,7.6
4,55,Bollène-Pierrelatte,0,INTERRÉGIONAL,9.5,9.5,9.4,9.4,9.6,9.5,...,10.7,10.3,9.9,9.0,11.0,9.9,9.9,10.1,10.1,9.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,9403,Calvi,94,CORSE,11.8,11.7,11.4,11.3,11.2,11.5,...,10.0,10.4,10.3,11.0,13.8,9.1,8.9,9.3,9.3,8.3
298,9404,Corte,94,CORSE,7.6,7.1,7.7,7.5,7.5,7.5,...,7.3,7.1,7.0,6.5,7.8,6.3,5.9,6.2,6.3,5.5
299,9405,Ghisonaccia,94,CORSE,10.3,9.5,9.9,10.3,10.3,10.2,...,9.0,8.8,8.4,7.3,9.6,7.7,7.4,7.7,7.9,7.1
300,9406,Porto-Vecchio,94,CORSE,9.8,9.8,9.9,10.0,10.0,10.0,...,9.1,10.3,10.1,10.8,13.6,9.1,9.1,9.1,8.5,7.7


In [33]:
chomage_df.dtypes

ZE2020         int64
LIBZE2020     object
REG            int64
LIBREG        object
2003-T1      float64
              ...   
2020-T4      float64
2021-T1      float64
2021-T2      float64
2021-T3      float64
2021-T4      float64
Length: 80, dtype: object

In [34]:
#modification du style des 2 colonnes avec code ZE pour pouvoir effectuer le MERGE
chomage_df['ZE2020'] = chomage_df['ZE2020'].astype(str)
INSEE1_df['code_ze'] = INSEE1_df['code_ze'].astype(str)

In [35]:
#MERGE : création de INSEE2_df en conctenant INSEE1_df avec les colonnes ZE2020, LIBZE2020 et 2021-T4 de chomage_df avec clé ZE2020 (numéro de ZE)
INSEE2_df = pd.merge(INSEE1_df, chomage_df, how = "left", left_on="code_ze", right_on="ZE2020")
INSEE2_df

,cp,code_commune,commune,code_commune2,commune2,code_ze,ze,ZE2020,LIBZE2020,REG,...,2019-T3,2019-T4,2020-T1,2020-T2,2020-T3,2020-T4,2021-T1,2021-T2,2021-T3,2021-T4
0,1000.0,01053,BOURG EN BRESSE,01053,Bourg-en-Bresse,8405,Bourg en Bresse,8405,Bourg en Bresse,84.0,...,6.2,6.1,5.8,5.6,6.9,6.0,6.0,5.9,6.0,5.5
1,1000.0,01053,BOURG EN BRESSE,01053,Bourg-en-Bresse,8405,Bourg en Bresse,8405,Bourg en Bresse,84.0,...,6.2,6.1,5.8,5.6,6.9,6.0,6.0,5.9,6.0,5.5
2,1000.0,01344,ST DENIS LES BOURG,01344,Saint-Denis-lès-Bourg,8405,Bourg en Bresse,8405,Bourg en Bresse,84.0,...,6.2,6.1,5.8,5.6,6.9,6.0,6.0,5.9,6.0,5.5
3,1090.0,01165,FRANCHELEINS,01165,Francheleins,8434,Villefranche-sur-Saône,8434,Villefranche-sur-Saône,84.0,...,6.4,6.3,6.0,5.8,7.0,6.2,6.2,6.1,6.1,5.5
4,1090.0,01165,FRANCHELEINS,01165,Francheleins,8434,Villefranche-sur-Saône,8434,Villefranche-sur-Saône,84.0,...,6.4,6.3,6.0,5.8,7.0,6.2,6.2,6.1,6.1,5.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39188,NaN,nan,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39189,NaN,nan,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39190,NaN,nan,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39191,NaN,nan,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
# supprime les colonnes inutiles cad 2003-T1 (colonne 5) à 2021-T3 (colonne 79)
# NE PARCHE PAS : INSEE2_df.drop(INSEE2_df.iloc[:,5:79],1,inplace=True)

In [37]:
# ne garder que les colonnes utiles
INSEE2_df = INSEE2_df[["cp", "code_commune", "commune", "code_commune2", "commune2", "code_ze", "ze", "ZE2020", "LIBZE2020", "2021-T4"]]
INSEE2_df.rename (columns = {'2021-T4' : 'chomage_2021T4'}, inplace = True)
INSEE2_df

C:\Users\emman\anaconda3\envs\projet_desu_tre\lib\site-packages\pandas\core\frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,cp,code_commune,commune,code_commune2,commune2,code_ze,ze,ZE2020,LIBZE2020,chomage_2021T4
0,1000.0,01053,BOURG EN BRESSE,01053,Bourg-en-Bresse,8405,Bourg en Bresse,8405,Bourg en Bresse,5.5
1,1000.0,01053,BOURG EN BRESSE,01053,Bourg-en-Bresse,8405,Bourg en Bresse,8405,Bourg en Bresse,5.5
2,1000.0,01344,ST DENIS LES BOURG,01344,Saint-Denis-lès-Bourg,8405,Bourg en Bresse,8405,Bourg en Bresse,5.5
3,1090.0,01165,FRANCHELEINS,01165,Francheleins,8434,Villefranche-sur-Saône,8434,Villefranche-sur-Saône,5.5
4,1090.0,01165,FRANCHELEINS,01165,Francheleins,8434,Villefranche-sur-Saône,8434,Villefranche-sur-Saône,5.5
...,...,...,...,...,...,...,...,...,...,...
39188,NaN,nan,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN
39189,NaN,nan,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN
39190,NaN,nan,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN
39191,NaN,nan,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN


In [38]:
#vérifier le nombre de valeurs manquantes (null ou NA) dans chaque colonne
INSEE2_df.isna().sum()

cp                267
code_commune        0
commune           267
code_commune2     324
commune2          324
code_ze             0
ze                324
ZE2020            372
LIBZE2020         372
chomage_2021T4    372
dtype: int64

In [39]:
# identification et visualisation des 48 lignes avec des valeurs manquantes
INSEE2_df[INSEE2_df.LIBZE2020.isna()]

,cp,code_commune,commune,code_commune2,commune2,code_ze,ze,ZE2020,LIBZE2020,chomage_2021T4
4670,13001.0,13201,MARSEILLE 01,NaN,NaN,nan,NaN,NaN,NaN,NaN
4671,13002.0,13202,MARSEILLE 02,NaN,NaN,nan,NaN,NaN,NaN,NaN
4672,13003.0,13203,MARSEILLE 03,NaN,NaN,nan,NaN,NaN,NaN,NaN
4673,13004.0,13204,MARSEILLE 04,NaN,NaN,nan,NaN,NaN,NaN,NaN
4674,13005.0,13205,MARSEILLE 05,NaN,NaN,nan,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
39188,NaN,nan,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN
39189,NaN,nan,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN
39190,NaN,nan,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN
39191,NaN,nan,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN


**Données manquantes**

> 48 lignes n°38374 à 38398 et n°38484 à 38506 : correspondent aux codes communes et ZE d'outremer 97XXX). Certaines sont en doublon.
> Suppression des lignes, en espérant que ces codes ne seront pas demandés dans le fichier "données internes".

In [40]:
# suppression des lignes avec des données manquantes
INSEE2_df = INSEE2_df.dropna()
INSEE_df

,code_commune2,commune2,code_ze,ze,cp,code_commune,commune
0,01001,L'Abergement-Clémenciat,8405,Bourg en Bresse,1400.0,01001,L ABERGEMENT CLEMENCIAT
1,01002,L'Abergement-de-Varey,8405,Bourg en Bresse,1640.0,01002,L ABERGEMENT DE VAREY
2,01004,Ambérieu-en-Bugey,8405,Bourg en Bresse,1500.0,01004,AMBERIEU EN BUGEY
3,01005,Ambérieux-en-Dombes,8434,Villefranche-sur-Saône,1330.0,01005,AMBERIEUX EN DOMBES
4,01006,Ambléon,8404,Belley,1300.0,01006,AMBLEON
...,...,...,...,...,...,...,...
38867,97614,Ouangani,601,Mayotte,97670.0,97614,OUANGANI
38868,97615,Pamandzi,601,Mayotte,97615.0,97615,PAMANDZI
38869,97616,Sada,601,Mayotte,97640.0,97616,SADA
38870,97617,Tsingoni,601,Mayotte,97680.0,97617,TSINGONI


In [41]:
# vérification de la nouvelle df
INSEE2_df.isna().sum()

cp                0
code_commune      0
commune           0
code_commune2     0
commune2          0
code_ze           0
ze                0
ZE2020            0
LIBZE2020         0
chomage_2021T4    0
dtype: int64

La nouvelle df comprend 38 872 lignes soit 54 de moins (auparavant 38926) et plus aucune valeur nulle
> POURQUOI 54 et NON 48 ??? et plus aucune valeur nulle

**Ajout de l'indice de concentration de l'emploi au JDD INSEE2 (constitution du JDD INSEE3)**

- Source : données INSEE publiées par l'Observatoire des Territories (agence nationale de la cohésion des territoire).

Dans un 2nd temps, on ajoute au JDD externe l'indice de concentration d'emploi par territoire communal.

> L'indice de concentration de l'emploi mesure le rapport entre le nombre d’emplois total proposés sur un territoire et le nombre d'actifs occupés (actifs en emploi) qui y résident. Cet indicateur permet d'apprécier la fonction de pôle d'emploi ou la fonction résidentielle d'un espace. Si cet indice est supérieur à 100 alors le nombre d'emplois proposés localement est plus important que le nombre d'actifs qui y résident et qui ont un emploi. Dans ce cas, le territoire considéré occupe une fonction de pôle d'emploi.
> *Le JDD en format Excel comprend l'indicateur de concentration d'emploi (nombre d'emplois pour 100 actifs occupés) pur chaque communes (périmètre 2021) sur une série temporelle allant de 1975 à 2018.*

- URL : https://www.observatoire-des-territoires.gouv.fr/indice-de-concentration-de-lemploi

Le fichier contient 2 feuilles : celle des données "Data" comprend 4 lignes de métadonnées à supprimer et seules les lignes correspondant à l'année 2018 nous intéresent (les lignes 1975, 1982, 1990, 1999, 2008 et 2013 peuvent être supprimées).

In [ ]:
# import du fichier avec suppression des métadonnées
emploi_df = pd.read_excel(r"C:\Users\emman\Documents\4_Diplômes\2022_DESU-AMU\0_Projet\projet_desu_tre\data\external\insee_rp_hist_1975.xlsx",  skiprows = 3, header = 1, sheet_name = 'Data')
emploi_df

In [ ]:
#droper les lignes correspondant aux années 1975, 1982, 1990, 1999, 2008 et 2013 et garder que l'année 2018
year_df_idx=emploi_df[emploi_df["an"]<2018].index
emploi_df = emploi_df.drop(year_df_idx)
emploi_df

Il ne reste que 34 965 colonnes équivalent aux années 2018 (suppression de 209790 lignes soit 85%)

In [ ]:
#modification du format de la colonne code géo pour pouvoir effectuer le MERGE
emploi_df['codgeo'] = emploi_df['codgeo'].astype(str)
#modification du format de la colonne "an" pour fixer l'année
emploi_df['an'] = emploi_df['an'].astype(str)

In [ ]:
#MERGE : création de INSEE3_df en conctenant INSEE2_df avec emploi_df avec clé codgeo (numéro de commune)
INSEE3_df = pd.merge(INSEE2_df, emploi_df, how = "left", left_on="code_commune2", right_on="codgeo")
INSEE3_df

Il y a le même nombre de lignes : 38 878.

In [ ]:
# vérifier les données manquantes de la nouvelle df
INSEE3_df.isna().sum()

In [ ]:
# identification et visualisation des 8 lignes avec des valeurs manquantes
INSEE3_df[INSEE3_df.ind_concentr_emp.isna()]

5 communes n'ont pas d'indicateur INDICE DE CONCENTRATION DE L'EMPLOI : si ces codes figurent dans la BDD des salariés, nous les supprimerons.

## 2. Consolidation de la BDD OPEN DATA

Il est maintenant possible de ne garder que les colonnes utiles pour réaliser le MERGE avec le JDD des salariés, à savoir :
- l'index
- le code postal (pour réaliser la jointure)
- le code commune et le code commune 2 (pour voir lequel permet la meilleure jointure)
- le taux de chômage
- l'indicateur de concentration d'emploi
- éventuellement le nom de la commune et celui de la zone d'emploi pour la partie descriptive

In [ ]:
# modification et préparation de la table INSEE3_df : en-têtes renommées
INSEE3_df = INSEE3_df.rename(columns = {"code_commune":'codgeo1', 
                                          "code_commune2" : 'codgeo2',  
                                          "codgeo" : 'codgeo3',
                                         "ind_concentr_emp" : 'concentr_emp2018'})
INSEE4_df = INSEE3_df[["cp", "codgeo1", "codgeo2", "codgeo3", "commune", "ze", "chomage_2021T4", "concentr_emp2018"]] # on ne garde que les colonnes utiles 
INSEE4_df

## 3. Import de la BDD endogène

La BDD des salariés a été constituée de manière fictive mais elle correspond à la typologie réelle d'une BDD comportant des datas sur le RETOUR A L'EMPLOI :
- le nom et prénom (ou matricule)
- l'âge, le genre (parfois l'ancienneté dans l'entreprise mais pas dans ce cas)
- la CSP (parfois l'intitulé métier et/ou le iveau de qualification, mais pas dans ce cas)
- le lieu de résidence (code postal, parfois le département)
- la taille et le secteur d'activité de l'entreprise d'origine
- le temps de retour à l'emploi (ou de solution trouvée)

Une fois cette BDD importée, on pourra y injecter les 2 indicateurs exogènes TAUX DE CHOMAGE et INDICE DE CONCENTRATION D'EMPLOI pour ajouter des données qualitatives à la BDD.

In [ ]:
# import de la BDD endogène RAE (Temps de Retour à l'Emploi)
tre_df = pd.read_csv (r"C:\Users\emman\Documents\4_Diplômes\2022_DESU-AMU\0_Projet\projet_desu_tre\data\raw\fake_RAE_v_3.csv")
tre_df

In [ ]:
#MERGE depuis le CODE GEOGRAPHIQUE (car le CODE POSTAL ne fonctionne pas)
df = pd.merge(tre_df, INSEE4_df, how = "left", left_on="code_commune", right_on="codgeo1")
df

In [ ]:
# vérifier les données manquantes de la nouvelle df
df.isna().sum()

**Traitement des doublons**

On est passé de 49 999 lignes à 148 049.
Il y'a donc plein de doublons, en plus des 2 252 lignes avec des données manquantes.

In [ ]:
df.nunique()

In [ ]:
# pour identifier les ligns en doublon (True = 1)
df.duplicated(subset=['id', 'prenom', 'nom'], keep=False)

In [ ]:
# Somme de la série pour pouvoir compter combien de valeurs sont dupliquées. 
# Le paramètre "first" au lieu de False permet d'identifier tous les doublons sauf la 1ère ligne que l'on veut garder
df.duplicated(subset=['id', 'prenom', 'nom'], keep="first").sum()

> On voit qu'il y a 122 867 doublons en tout mais 98 050 si l'on garde la première occurence, ce qui correspond exactement à la taille de la BDD endogène d'origine. On va donc pouvoir procéder à l'élimination de ces doublons.

In [ ]:
# suppresion des doublons
df_unique=df.drop_duplicates(subset =['id', 'prenom', 'nom'],keep="first") 
df_unique

In [ ]:
# vérifier les données manquantes de la nouvelle df
df_unique.isna().sum()

In [ ]:
# données techniques sur la df
df_unique.info()

> Nous disposons désormais d'une BDD de 49 999 individus avec un id spécifique, 2252 lignes ayant des données manquantes, et 23 colonnes.

## 4. Finalisation de la BDD

Pour finaliser le JDD, nous allons pouvoir supprimer :
- les lignes avec des données manquantes
- les colonnes inutiles : date_de_naissance (l'age suffira), le code NAF (la répartition en 88 NAF est trop compliquée à gérer en ONE HOT ENCODING donc on l'élimine pour cette fois), le département, la population totale, le cp et les 3 codgeo, la commune (on ne garde que la ville).

Nous pourrons ensuite l'exporter pour l'utiliser dans le prochain notebook consacré à la statistique descriptive.

In [ ]:
# suppression des colonnes inutiles
df2 = df_unique[["id", "genre", "prenom", "nom", "age", "csp", "nbr_de_salaries", "ville", "duree_rae", "ze", "chomage_2021T4", "concentr_emp2018"]] # on ne garde que les colonnes utiles 
df2

In [ ]:
# suppression des lignes avec des données manquantes
df2 = df2.dropna()
df2

On a réduit la dataframe de 23 à 12 colonnes et de 49999 à 47747 lignes correspondant à la suppression des 2 252 lignes avec des données manquantes.

In [ ]:
# vérification qu'il n'y a plus aucune donné manquante
df2.isna().sum()

In [ ]:
# export de la dataframe afin de la réimporter dans la notebook consacré à la statistique descriptive.
df2.to_csv(r'C:\Users\emman\Documents\4_Diplômes\2022_DESU-AMU\0_Projet\projet_desu_tre\data\processed\tre_dataframe.csv', index = False)

La dataframe est désormais propre (pas de données manquantes) et elle comprend uniquement les indicateurs souhaités pour l'analyse, au bon format.
Nous avons retiré les colonnes d'identification de type CP ou code géographiques qui pourraient être considérés comme des données numériques, ainsi que la colonne catégorielle NAF-APE trop complexe à traiter (OHE).